# LUVOIR A Simulator Demo

In [ ]:
import os
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import hcipy
from astropy.io import fits

from pastis.config import CONFIG_PASTIS
import pastis.util
from pastis.e2e_simulators.luvoir_imaging import LuvoirA_APLC

matplotlib.rc('image', origin='lower')    # Make sure image origin is always in lower left

In [ ]:
# Define sampling and set path for LUVOIR A input files (they are on the repo)
sampling = CONFIG_PASTIS.getfloat('LUVOIR', 'sampling')
optics_input = os.path.join(pastis.util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))

In [ ]:
# Pick which design you want (small, medium, large) and instantiate LUVOIR A simulator object
design = 'small'
luvoir = LuvoirA_APLC(optics_input, design, sampling)

This will instantiate LUOVIR A with the chosen APLC, and also a simple segmented DM that can do piston, tip and tilt on each segment only.

In [ ]:
# Calculate a coronagraphic and a direct PSF, display all intermediate planes while doing so
coro, direct = luvoir.calc_psf(display_intermediate=True, ref=True)

In [ ]:
# We have a LOWFS
lowfs_im = luvoir.calc_low_order_wfs()
plt.imshow(lowfs_im.intensity.shaped)

In [ ]:
# We have a OBWFS
obwfs_im = luvoir.calc_out_of_band_wfs()
plt.imshow(obwfs_im.intensity.shaped)

## Creating, commanding and deleting the various deformable mirrors

You can run the individual cells below to add different deformable mirrors:
1. Full segmented mirror with the capability for an arbitrary number of local Zernikes
2. Segmented DM for Harris modes as local segment modes
3. Global Zernike DM
4. DM for Fourier ripples
5. Continuous deformable mirror

When you don't actually need a DM, do not create it as it will only slow down the propagations. If you needed to create it but now don't need it any more, you can use the appropriate methods to remove it fro your simulator instance.

### 1. Multi-mode segmented DM (Zernikes)

In [ ]:
# Create multi mode segmented mirror
# !! THIS CELL TAKES QUITE A WHILE TO RUN !!
n_modes_segs = 3
luvoir.create_segmented_mirror(n_modes_segs)

In [ ]:
# Command the DM
new_command = np.zeros(120*n_modes_segs)
new_command[4] = 2e-8
new_command[51] = 2e-4
new_command[346] = 2e-8
luvoir.sm.actuators = new_command

In [ ]:
# Remove the DM from simulator instance
luvoir.remove_segmented_mirror()

When you remove this multi-mode segmented DM, you go back to having the simple piston, tip, tilt segmented DM.

### 2. Harris modes segmented DM

In [ ]:
# Create segmented Harris mode mirror
# !! THIS CELL TAKES QUITE A WHILE TO RUN !!
fpath = '/Users/ilaginja/repos/PASTIS/Sensitivities2.xlsx'    # path to Harris spreadsheet
pad_orientations = np.pi / 2 * np.ones(120)
luvoir.create_segmented_harris_mirror(fpath, pad_orientations)

In [ ]:
# Command the DM
new_command = np.zeros(luvoir.harris_sm.num_actuators)
new_command[18] = 1e-8
new_command[37] = 2e-8
luvoir.harris_sm.actuators = new_command

In [ ]:
# Remove the DM from simulator instance
luvoir.remove_segmented_harris_mirror()

### 3. Global Zernike DM (global low-order modes)

In [ ]:
# Create low-order (Zernike) mode mirror
n_modes_zernikes = 15
luvoir.create_global_zernike_mirror(n_modes_zernikes)

In [ ]:
# Command the DM
new_command = np.zeros(n_modes_zernikes)
new_command[7] = 2e-8
luvoir.zernike_mirror.actuators = new_command

In [ ]:
# Remove the DM from simulator instance
luvoir.remove_global_zernike_mirror()

### 4. Ripple DM (global high-order modes)

In [ ]:
# Create high-order (ripple) mode mirror
n_ripples = 5    # need to use odd number
luvoir.create_ripple_mirror(n_ripples)

In [ ]:
# Command the DM
new_command = np.zeros(n_ripples*n_ripples)
new_command[12] = 2e-8
luvoir.ripple_mirror.actuators = new_command

In [ ]:
# Remove the DM from simulator instance
luvoir.remove_ripple_mirror()

### 5. Continuous DM

In [ ]:
# Create a good ol' continuous DM
n_acts_across = 15 
luvoir.create_continuous_deformable_mirror(n_acts_across)

In [ ]:
# Command the DM
new_command = np.zeros(n_acts_across*n_acts_across)
new_command[66] = 2e-8
new_command[77] = 2e-8
luvoir.dm.actuators = new_command

In [ ]:
# Remove the DM from simulator instance
luvoir.remove_continuous_deformable_mirror()

## Normalization to one photon

All `calc_psf()` methods have a parameter `norm_one_photon` that allows you to normalize the propagations to one photon. The default is set to `None` and you have to actively set it to `norm_one_photon=True` wen you need it.

In [ ]:
# Calculate a coronagraphic and a direct PSF, display all intermediate planes while doing so
coro, direct = luvoir.calc_psf(display_intermediate=True, ref=True, norm_one_photon=True)